In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pymssql

#### Connect to the SQL server

In [2]:
server = 'fs.rice.edu'
database = 'stocks'
username = 'stocks'
password = '6LAZH1'

string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database 
conn = create_engine(string).connect()

#### Read some GHZ predictors

In [12]:
df = pd.read_sql("select ticker, roeq, acc, agr, mom12m, mom1m, price, mve from today", conn)
df = df.dropna()
df = df[df.price>=5]

df = df.set_index("ticker")


#### Choose desired size category

This filters to stocks outside top 1000.

In [13]:
df["size"] = df.mve.rank(ascending=False)
df = df[df["size"]>1000].copy()

#### Rank on predictors, 0=worst, 1=best

Assume high roeq and mom12m are best.  Assume low acc, agr, and mom1m are best.

In [18]:
for predictor in ["roeq", "mom12m"]:
    df[predictor] = df[predictor].rank(pct=True, ascending=False)
for predictor in ["acc", "agr", "mom1m"]:
    df[predictor] = df[predictor].rank(pct=True, ascending=False)
df["composite"] = df[["roeq", "mom12m", "acc", "agr", "mom1m"]].mean(axis=1)
df = df.sort_values(by="composite", ascending=False)
df.iloc[:100].to_csv("files/top.csv")
